<a href="https://colab.research.google.com/github/MariMurotani/ColabNotebooks/blob/main/CFWD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/CFWD

/content/drive/MyDrive/Colab Notebooks/CFWD


In [ ]:
!git clone https://github.com/He-Jinhong/CFWD.git

fatal: destination path 'CFWD' already exists and is not an empty directory.


In [ ]:
%cd CFWD

/content/drive/MyDrive/Colab Notebooks/CFWD/CFWD


In [ ]:
# NOTE: 単語のペアのモデルと2種類あるからどっちかよく確認した方がいい
#!gdown --id 1KaeDDTPQWB9YgFjbYFmO9H_5hKrHTCNd -O pretrain_models/CFWD_LOLv1.pth
!gdown --id 1dmu3udLbBKSuD26McYFPeqkNGEX4IuE1 -O pretrain_models/CFWD_LOLv1.pth

# https://drive.google.com/file/d/1KaeDDTPQWB9YgFjbYFmO9H_5hKrHTCNd/view
# https://drive.google.com/file/d/1KaeDDTPQWB9YgFjbYFmO9H_5hKrHTCNd/view
# https://drive.google.com/file/d/1dmu3udLbBKSuD26McYFPeqkNGEX4IuE1/view

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1dmu3udLbBKSuD26McYFPeqkNGEX4IuE1
From (redirected): https://drive.google.com/uc?id=1dmu3udLbBKSuD26McYFPeqkNGEX4IuE1&confirm=t&uuid=c774ffce-6dd1-4cf9-8362-82ac56b13ddb
To: /content/drive/MyDrive/Colab Notebooks/CFWD/CFWD/pretrain_models/CFWD_LOLv1.pth
100% 354M/354M [00:02<00:00, 118MB/s] 


In [ ]:
# ❌ Colabプリインストール版を完全に削除
!pip uninstall -y numpy tensorboard tensorboard-data-server tb-nightly

In [ ]:
# ✅ 安定バージョンを明示的にインストール
!pip install numpy==1.24.4
!pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118
!pip install git+https://github.com/openai/CLIP.git
!pip install einops timm==0.9.2 opencv-python scikit-image PyWavelets pytorch-msssim ftfy Pillow==9.5.0

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 126.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89990 sha256=c7e74ca39046811447436076e53c0979a35e3d05bfbed8407e6e6f64e81c7d95
  Stored in directory: /root/.cache/pip/wheels/fc/5d/45/34fe9945d5e45e261134e72284395be36c2d4828af38e2b0fe
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cu124
    Uninstalling torch-2.6.0+cu124:
      Successfully uninstalled torch

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/CFWD/CFWD

/content/drive/MyDrive/Colab Notebooks/CFWD/CFWD


In [ ]:
from models.ddm import DenoisingDiffusion
import torch
from PIL import Image
import torchvision.transforms as T
import yaml
import argparse
import os

In [ ]:
def dict2namespace(config_dict):
    namespace = argparse.Namespace()
    for k, v in config_dict.items():
        setattr(namespace, k, dict2namespace(v) if isinstance(v, dict) else v)
    return namespace

In [ ]:
# 推論用モデルの準備
args = argparse.Namespace(resume='pretrain_models/CFWD_LOLv1.pth', sampling_timesteps=10, image_folder='')
# YAMLファイル読み込み
with open('configs/LOLv1.yml', 'r') as f:
    config_dict = yaml.safe_load(f)

config = dict2namespace(config_dict)
config.device = torch.device("cuda")

model = DenoisingDiffusion(args, config)
model.load_ddm_ckpt(args.resume, ema=True)
model.model.eval()

Load checkpoint:  True
Current checkpoint: pretrain_models/CFWD_LOLv1.pth


DataParallel(
  (module): Net(
    (high_enhance0): HFRM(
      (conv_head): Depth_conv(
        (depth_conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3)
        (point_conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
      )
      (dilated_block_LH): Dilated_Resblock(
        (model): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): LeakyReLU(negative_slope=0.01)
          (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
          (3): LeakyReLU(negative_slope=0.01)
          (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), dilation=(3, 3))
          (5): LeakyReLU(negative_slope=0.01)
          (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
          (7): LeakyReLU(negative_slope=0.01)
          (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
     

In [ ]:
# 入力画像読み込み・前処理
image_path = "/content/drive/MyDrive/Colab Notebooks/Data/Panorama/living_dark_768.jpg"
image_path = "/content/drive/MyDrive/Colab Notebooks/Data/dark_normal_picture.jpg"
image = Image.open(image_path).convert("RGB")
print(image.size)

# T.ToTensor()	PIL画像（H×W×Cの範囲0〜255）を、PyTorch用のテンソル（C×H×W、値は0.0〜1.0）に変換
# T.ToTensor()(image)	image をテンソル化（例：torch.Size([3, 768, 1536])）
# .unsqueeze(0)	ミニバッチ次元（B=1）を追加 → torch.Size([1, 3, 768, 1536])
# .cuda()	テンソルを GPU（CUDAデバイス）に転送
input_tensor = T.ToTensor()(image).unsqueeze(0).cuda()

# 推論
with torch.no_grad():
    output = model.model(input_tensor)
    restored_image = output["pred_x"]

# 保存
basename = os.path.basename(image_path)
name, _ = os.path.splitext(basename)
output_path = f"{name}_restored.jpg"

# restored_image	モデル出力（shape: [1, 3, H, W]）
# .squeeze(0)	バッチ次元 1 を削除 → [3, H, W]
# .cpu()	GPU→CPU にテンソルを移動（保存処理などで必要）
# .clamp(0, 1)	値を 0〜1の範囲に制限（画像なのでこの範囲を超えると不正になるため）
T.ToPILImage()(restored_image.squeeze(0).cpu().clamp(0, 1)).save(output_path)
print(f"保存完了: {output_path}")

(992, 576)
保存完了: dark_normal_picture_restored.jpg
